In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

### Load the data

In [2]:
data = pd.read_csv("../data/alzheimers_disease_data.csv")
data = data.drop("DoctorInCharge", axis=1) # this attribute is confidential in the data, and thus not useful 
data.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,6.518877,0,0,1.725883,0,0,0,1,0,0
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,7.118696,0,0,2.592424,0,0,0,0,1,0
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,5.895077,0,0,7.119548,0,1,0,1,0,0
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,8.965106,0,1,6.481226,0,0,0,0,0,0
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,6.045039,0,0,0.014691,0,0,1,1,0,0


### Oversampling by ethnicity

In [3]:
e = data["Ethnicity"]
print('Original dataset shape %s' % Counter(e))

ethnicity_counts = dict(data["Ethnicity"].value_counts())
num_ethnicities = len(ethnicity_counts)
max_count = max(ethnicity_counts.values())

strategy_over = {ethnicity: max_count for ethnicity in range(num_ethnicities)}
over = SMOTE(sampling_strategy=strategy_over)
print(strategy_over, sep='\n')

data_over, e_over=over.fit_resample(data, e)
print('Oversampled dataset shape %s' % Counter(e_over))

Original dataset shape Counter({0: 1278, 1: 454, 3: 211, 2: 206})
{0: np.int64(1278), 1: np.int64(1278), 2: np.int64(1278), 3: np.int64(1278)}
Oversampled dataset shape Counter({0: 1278, 3: 1278, 1: 1278, 2: 1278})


### Split the data

In [4]:
X, y = data_over.drop("Diagnosis", axis=1), data_over["Diagnosis"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22, stratify=y)
X_train.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,MMSE,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness
290,5041,69,1,2,2,17.630318,0,2.720553,0.997862,1.886614,...,13.580760,7.555930,0,0,6.898207,0,1,0,0,0
5043,4856,82,0,3,1,31.911154,0,12.723206,2.245509,6.332117,...,10.829459,5.028921,0,0,4.991064,0,0,0,0,1
2083,6834,66,0,3,0,21.990350,0,12.820507,5.350639,7.145357,...,15.333487,8.826886,0,0,5.227759,1,0,0,0,0
2580,5754,73,0,1,2,20.464414,0,5.307350,4.188961,3.463421,...,17.512040,6.670895,0,0,2.363409,0,0,0,0,0
4901,4811,80,0,3,1,39.220902,0,16.258472,8.794156,6.862098,...,16.994199,7.957731,0,0,7.418498,0,1,0,0,0


### Train the model

In [5]:
classifier = MLPClassifier(max_iter=5000, random_state=12)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [6]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.09      0.17       986
           1       0.25      1.00      0.39       292

    accuracy                           0.30      1278
   macro avg       0.62      0.55      0.28      1278
weighted avg       0.83      0.30      0.22      1278



In [ ]:
X["Ethnicity"].value_counts()

Ethnicity
0    1278
1     454
3     211
2     206
Name: count, dtype: int64

### Compare performance based on ethnicity

In [8]:
# Join X_test, y_test, y_pred for analysis
results = X_test.copy()
results["TrueDiagnosis"] = y_test
results["PredictedDiagnosis"] = y_pred

In [9]:
# Ethnicity 0
eth0_results = results[results["Ethnicity"] == 0]
print(classification_report(eth0_results["TrueDiagnosis"], eth0_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       1.00      0.04      0.08       210
           1       0.36      1.00      0.52       111

    accuracy                           0.37       321
   macro avg       0.68      0.52      0.30       321
weighted avg       0.78      0.37      0.24       321



In [10]:
# Ethncity 1
eth1_results = results[results["Ethnicity"] == 1]
print(classification_report(eth1_results["TrueDiagnosis"], eth1_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       1.00      0.08      0.15       246
           1       0.21      1.00      0.35        60

    accuracy                           0.26       306
   macro avg       0.60      0.54      0.25       306
weighted avg       0.85      0.26      0.19       306



In [11]:
# Ethnicity 2  
eth2_results = results[results["Ethnicity"] == 2]
print(classification_report(eth2_results["TrueDiagnosis"], eth2_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       1.00      0.09      0.16       235
           1       0.27      1.00      0.43        80

    accuracy                           0.32       315
   macro avg       0.64      0.54      0.29       315
weighted avg       0.81      0.32      0.23       315



In [12]:
# Ethncity 3
eth3_results = results[results["Ethnicity"] == 3]
print(classification_report(eth3_results["TrueDiagnosis"], eth3_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       1.00      0.14      0.24       295
           1       0.14      1.00      0.24        41

    accuracy                           0.24       336
   macro avg       0.57      0.57      0.24       336
weighted avg       0.89      0.24      0.24       336

